# Lab 6.3: Respiratory Performance Across Environments
## Chapter 6: The Oxygen Revolution - Respiratory Systems

### 🎯 Learning Objectives
- Analyze respiratory performance across extreme environments
- Calculate oxygen availability at altitude and during dives
- Compare high-altitude and deep-diving adaptations
- Understand physiological limits and performance envelopes
- Apply principles to real-world challenges

### 📖 Connection to Chapter 6
**Sections 6.3-6.6**: Lung evolution, bird respiratory systems, diving adaptations, high-altitude breathing

### 🌍 The Challenge
From 2,000m ocean depths to 8,000m mountain peaks - vertebrates conquer extreme respiratory challenges!

In [ ]:
# === GOOGLE COLAB SETUP - RUN THIS FIRST ===
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    print("✓ Widgets enabled for Google Colab")
except:
    print("✓ Running outside Colab")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact, FloatSlider, Dropdown, Button, Output, HTML, IntSlider, HBox
from IPython.display import display
from datetime import datetime
import os

print("✓ Libraries loaded! Ready to explore extreme environments...")

## Part 1: Environmental Physics

### Altitude Effects on Oxygen
- **Sea level**: 160 mmHg O₂ partial pressure
- **2,500m**: Human altitude sickness threshold
- **5,500m**: Permanent habitation limit
- **8,000m**: Bar-headed goose migration altitude
- **8,848m (Everest)**: ~43 mmHg (27% of sea level)

### Key Equation
$$P_{altitude} = P_{sea\_level} \times e^{-altitude/7400}$$

In [ ]:
# Environmental calculation functions
def calculate_altitude_oxygen(altitude_m, sea_level_po2=160):
    """Calculate O2 partial pressure at altitude using barometric equation"""
    scale_height = 7400  # meters
    po2 = sea_level_po2 * np.exp(-altitude_m / scale_height)
    return po2

def calculate_dive_oxygen(dive_time_min, body_mass_kg, metabolic_rate=1.0, o2_stores=50):
    """Calculate oxygen depletion during dive"""
    # Basal VO2 = 10 × Mass^0.75 ml/min
    basal_vo2 = 10 * (body_mass_kg ** 0.75)
    actual_vo2 = basal_vo2 * metabolic_rate
    
    # Total O2 stores: ~20 ml/kg × stores_factor
    total_o2 = 20 * body_mass_kg * (o2_stores / 50)
    
    # Oxygen over time
    time_points = np.linspace(0, dive_time_min, 100)
    o2_remaining = total_o2 - (actual_vo2 * time_points)
    o2_remaining = np.maximum(o2_remaining, 0)
    
    # Aerobic limit (20% remaining)
    aerobic_limit = (total_o2 * 0.8) / actual_vo2 if actual_vo2 > 0 else dive_time_min
    
    return {
        'time': time_points,
        'o2_ml': o2_remaining,
        'o2_percent': (o2_remaining / total_o2) * 100,
        'aerobic_limit': aerobic_limit,
        'total_o2': total_o2,
        'vo2': actual_vo2
    }

# Test
everest_o2 = calculate_altitude_oxygen(8848)
print(f"Mt. Everest O₂: {everest_o2:.1f} mmHg ({everest_o2/160*100:.0f}% of sea level)")

whale = calculate_dive_oxygen(60, 35000, 0.3, 90)
print(f"Sperm whale 60-min dive: {whale['o2_percent'][-1]:.0f}% O₂ remaining")
print("\n✓ Calculation functions ready!")

## Part 2: Species Database

Real data from Chapter 6 case studies:

In [ ]:
# Species database with Chapter 6 data
species_db = {
    'Bar-Headed Goose': {
        'type': 'altitude',
        'max_altitude': 8800,
        'mass_kg': 2.5,
        'hb_affinity': 1.25,
        'efficiency': 90,
        'description': 'Migrates over Himalayas at 8,000+ meters. Enhanced hemoglobin, '
                      'unidirectional airflow, maintains flight metabolism at extreme altitude.'
    },
    'Vicuña': {
        'type': 'altitude',
        'max_altitude': 5500,
        'mass_kg': 50,
        'hb_affinity': 1.15,
        'efficiency': 35,
        'description': 'Lives year-round at 3,200-5,500m in Andes. Larger lungs, '
                      'more red blood cells, enhanced hemoglobin.'
    },
    'Human (Tibetan)': {
        'type': 'altitude',
        'max_altitude': 5000,
        'mass_kg': 60,
        'hb_affinity': 1.05,
        'efficiency': 25,
        'description': 'Genetic adaptations after 30,000 years. Enhanced nitric oxide, '
                      'increased blood flow, normal RBC count.'
    },
    'Human (Sea Level)': {
        'type': 'altitude',
        'max_altitude': 2500,
        'mass_kg': 70,
        'hb_affinity': 1.0,
        'efficiency': 25,
        'description': 'Baseline. Altitude sickness above 2,500m. '
                      'Requires weeks of acclimatization.'
    },
    'Sperm Whale': {
        'type': 'diving',
        'max_depth': 2000,
        'max_duration': 90,
        'mass_kg': 35000,
        'o2_stores': 90,
        'metabolic_rate': 0.3,
        'myoglobin': 8.0,
        'description': 'Deepest diver. 90% O₂ in blood/muscle. Extreme bradycardia, '
                      'metabolic suppression, hunts giant squid at 1,000-2,000m.'
    },
    'Weddell Seal': {
        'type': 'diving',
        'max_depth': 600,
        'max_duration': 82,
        'mass_kg': 450,
        'o2_stores': 75,
        'metabolic_rate': 0.4,
        'myoglobin': 6.0,
        'description': 'Antarctic specialist. Can dive 82 minutes. Switches to anaerobic '
                      'metabolism when O₂ depleted.'
    },
    'Emperor Penguin': {
        'type': 'diving',
        'max_depth': 565,
        'max_duration': 27,
        'mass_kg': 30,
        'o2_stores': 70,
        'metabolic_rate': 0.5,
        'myoglobin': 5.0,
        'description': 'Deepest-diving bird. Combines avian respiratory efficiency '
                      'with diving adaptations. 20+ dives per hour while foraging.'
    },
    'Human (Free Diver)': {
        'type': 'diving',
        'max_depth': 214,
        'max_duration': 11.5,
        'mass_kg': 70,
        'o2_stores': 50,
        'metabolic_rate': 0.7,
        'myoglobin': 1.0,
        'description': 'World record holders achieve remarkable feats but severely '
                      'limited vs marine mammals. Training enables bradycardia.'
    }
}

print("SPECIES DATABASE LOADED")
print("=" * 60)
altitude_sp = [n for n, d in species_db.items() if d['type'] == 'altitude']
diving_sp = [n for n, d in species_db.items() if d['type'] == 'diving']
print(f"\nAltitude Specialists ({len(altitude_sp)}):")
for sp in altitude_sp:
    print(f"  • {sp}")
print(f"\nDiving Specialists ({len(diving_sp)}):")
for sp in diving_sp:
    print(f"  • {sp}")
print("\n✓ Database ready for analysis!")

## Part 3: Altitude Performance Explorer

In [ ]:
# Altitude performance analyzer
def analyze_altitude(species_name, altitude):
    """Analyze species performance at altitude"""
    sp = species_db[species_name]
    
    # O2 availability
    o2_available = calculate_altitude_oxygen(altitude)
    o2_percent = (o2_available / 160) * 100
    effective_o2 = o2_available * sp['hb_affinity']
    
    # Performance calculation
    max_alt = sp['max_altitude']
    if altitude > max_alt * 1.5:
        performance = 0
        status = "Lethal"
        color = "darkred"
    elif altitude > max_alt:
        performance = 100 * np.exp(-((altitude - max_alt) / 1000))
        status = "Critical" if performance < 20 else "Struggling"
        color = "red" if performance < 20 else "orange"
    else:
        performance = 100 - (altitude / max_alt) * 25
        status = "Optimal" if performance > 80 else "Functional"
        color = "green" if performance > 80 else "yellow"
    
    performance = np.clip(performance, 0, 100)
    
    # Create visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Performance Capacity', 'Oxygen Availability', 
                       'Performance vs Altitude', 'Adaptations'),
        specs=[[{'type': 'indicator'}, {'type': 'indicator'}],
               [{'type': 'scatter'}, {'type': 'bar'}]]
    )
    
    # Performance gauge
    fig.add_trace(go.Indicator(
        mode="gauge+number",
        value=performance,
        title={'text': f"{status}"},
        number={'suffix': "%"},
        gauge={
            'axis': {'range': [0, 100]},
            'bar': {'color': color},
            'steps': [
                {'range': [0, 20], 'color': '#FFE5E5'},
                {'range': [20, 50], 'color': '#FFE1CC'},
                {'range': [50, 80], 'color': '#FFF8DC'},
                {'range': [80, 100], 'color': '#D4F1E8'}
            ]
        }
    ), row=1, col=1)
    
    # O2 gauge
    fig.add_trace(go.Indicator(
        mode="gauge+number",
        value=o2_percent,
        title={'text': "O₂ vs Sea Level"},
        number={'suffix': "%"},
        gauge={'axis': {'range': [0, 100]}, 'bar': {'color': '#0077B6'}}
    ), row=1, col=2)
    
    # Performance envelope
    alts = np.linspace(0, 10000, 100)
    perfs = []
    for a in alts:
        if a > max_alt * 1.5:
            perfs.append(0)
        elif a > max_alt:
            perfs.append(100 * np.exp(-((a - max_alt) / 1000)))
        else:
            perfs.append(100 - (a / max_alt) * 25)
    
    fig.add_trace(go.Scatter(
        x=alts, y=perfs,
        fill='tozeroy',
        line=dict(color='#2A9D8F', width=3),
        fillcolor='rgba(42, 157, 143, 0.3)',
        name='Capacity'
    ), row=2, col=1)
    
    fig.add_trace(go.Scatter(
        x=[altitude], y=[performance],
        mode='markers',
        marker=dict(size=15, color='red', symbol='diamond'),
        name='Current'
    ), row=2, col=1)
    
    # Adaptations
    adapt_scores = {
        'Efficiency': sp['efficiency'],
        'Hemoglobin': (sp['hb_affinity'] - 0.8) * 100,
        'Altitude Max': (sp['max_altitude'] / 10000) * 100
    }
    
    fig.add_trace(go.Bar(
        x=list(adapt_scores.keys()),
        y=list(adapt_scores.values()),
        marker_color=['#F4A261', '#E76F51', '#2A9D8F'],
        text=[f"{v:.0f}" for v in adapt_scores.values()],
        textposition='auto'
    ), row=2, col=2)
    
    fig.update_xaxes(title_text="Altitude (m)", row=2, col=1)
    fig.update_yaxes(title_text="Performance (%)", row=2, col=1)
    fig.update_yaxes(title_text="Score", row=2, col=2)
    
    fig.update_layout(
        height=700,
        title_text=f"<b>{species_name} at {altitude:,}m Altitude</b>",
        showlegend=False
    )
    
    # Print summary
    print("\n" + "="*60)
    print(f"ALTITUDE ANALYSIS: {species_name}")
    print("="*60)
    print(f"Altitude: {altitude:,}m")
    print(f"O₂ Available: {o2_available:.1f} mmHg ({o2_percent:.0f}% sea level)")
    print(f"Effective O₂: {effective_o2:.1f} mmHg (with {sp['hb_affinity']:.2f}× Hb affinity)")
    print(f"Performance: {performance:.0f}% - {status}")
    print(f"Max Altitude: {max_alt:,}m")
    print(f"\nDescription: {sp['description']}")
    print("="*60)
    
    fig.show()
    
    return {'species': species_name, 'altitude': altitude, 
            'performance': performance, 'status': status}

# Create interface
altitude_species = [n for n, d in species_db.items() if d['type'] == 'altitude']

species_alt = Dropdown(
    options=altitude_species,
    value='Bar-Headed Goose',
    description='Species:',
    style={'description_width': '100px'}
)

altitude_slider = IntSlider(
    value=5000,
    min=0,
    max=10000,
    step=500,
    description='Altitude (m):',
    style={'description_width': '100px'}
)

display(HTML("<h3>🏔️ Altitude Performance Explorer</h3>"))
interact(analyze_altitude, species_name=species_alt, altitude=altitude_slider);

## Part 4: Diving Performance Explorer

In [ ]:
# Diving performance analyzer
def analyze_dive(species_name, dive_duration):
    """Analyze diving performance"""
    sp = species_db[species_name]
    
    # Calculate O2 depletion
    result = calculate_dive_oxygen(
        dive_duration, 
        sp['mass_kg'],
        sp['metabolic_rate'],
        sp['o2_stores']
    )
    
    # Status
    if dive_duration <= result['aerobic_limit']:
        status = "Aerobic"
        color = "green"
    elif dive_duration <= sp['max_duration']:
        status = "Anaerobic"
        color = "orange"
    else:
        status = "Exceeds Capacity"
        color = "red"
    
    # Create visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Oxygen Depletion', 'O₂ Remaining', 
                       'Duration Comparison', 'Storage Capacity'),
        specs=[[{'type': 'scatter'}, {'type': 'indicator'}],
               [{'type': 'bar'}, {'type': 'bar'}]]
    )
    
    # O2 depletion curve
    fig.add_trace(go.Scatter(
        x=result['time'],
        y=result['o2_percent'],
        fill='tozeroy',
        line=dict(color='#0077B6', width=3),
        fillcolor='rgba(0, 119, 182, 0.3)',
        name='O₂'
    ), row=1, col=1)
    
    # Mark limits
    fig.add_vline(x=result['aerobic_limit'], line_dash="dash", 
                 line_color="orange", annotation_text="Aerobic Limit",
                 row=1, col=1)
    fig.add_vline(x=dive_duration, line_color="red",
                 annotation_text="Current", row=1, col=1)
    
    # O2 remaining gauge
    o2_left = result['o2_percent'][-1]
    fig.add_trace(go.Indicator(
        mode="gauge+number",
        value=o2_left,
        title={'text': status},
        number={'suffix': "%"},
        gauge={
            'axis': {'range': [0, 100]},
            'bar': {'color': color},
            'steps': [
                {'range': [0, 20], 'color': '#FFE5E5'},
                {'range': [20, 50], 'color': '#FFF8DC'},
                {'range': [50, 100], 'color': '#D4F1E8'}
            ]
        }
    ), row=1, col=2)
    
    # Duration bars
    durations = {
        'Current': dive_duration,
        'Aerobic': result['aerobic_limit'],
        'Maximum': sp['max_duration']
    }
    
    fig.add_trace(go.Bar(
        x=list(durations.keys()),
        y=list(durations.values()),
        marker_color=['red', 'orange', 'green'],
        text=[f"{v:.1f}" for v in durations.values()],
        textposition='auto'
    ), row=2, col=1)
    
    # Storage comparison
    storage = {
        species_name: sp['o2_stores'],
        'Terrestrial': 50
    }
    
    fig.add_trace(go.Bar(
        x=list(storage.keys()),
        y=list(storage.values()),
        marker_color=['#2A9D8F', '#999999'],
        text=[f"{v}%" for v in storage.values()],
        textposition='auto'
    ), row=2, col=2)
    
    fig.update_xaxes(title_text="Time (min)", row=1, col=1)
    fig.update_yaxes(title_text="O₂ Remaining (%)", row=1, col=1)
    fig.update_yaxes(title_text="Duration (min)", row=2, col=1)
    fig.update_yaxes(title_text="Storage (%)", row=2, col=2)
    
    fig.update_layout(
        height=700,
        title_text=f"<b>{species_name} - {dive_duration:.1f} Minute Dive</b>",
        showlegend=False
    )
    
    # Print summary
    print("\n" + "="*60)
    print(f"DIVE ANALYSIS: {species_name}")
    print("="*60)
    print(f"Dive Duration: {dive_duration:.1f} minutes")
    print(f"Aerobic Limit: {result['aerobic_limit']:.1f} minutes")
    print(f"Maximum: {sp['max_duration']:.1f} minutes")
    print(f"O₂ Remaining: {o2_left:.1f}% - {status}")
    print(f"Total Stores: {result['total_o2']:.0f} ml")
    print(f"Consumption: {result['vo2']:.1f} ml/min")
    print(f"\nDescription: {sp['description']}")
    print("="*60)
    
    fig.show()
    
    return {'species': species_name, 'duration': dive_duration,
            'o2_remaining': o2_left, 'status': status}

# Create interface
diving_species = [n for n, d in species_db.items() if d['type'] == 'diving']

species_dive = Dropdown(
    options=diving_species,
    value='Sperm Whale',
    description='Species:',
    style={'description_width': '100px'}
)

duration_slider = FloatSlider(
    value=30,
    min=1,
    max=120,
    step=5,
    description='Duration (min):',
    style={'description_width': '100px'}
)

display(HTML("<h3>🐋 Diving Performance Explorer</h3>"))
interact(analyze_dive, species_name=species_dive, dive_duration=duration_slider);

## Part 5: Multi-Species Comparisons

In [ ]:
def compare_altitude_species():
    """Compare all altitude specialists"""
    species_list = ['Bar-Headed Goose', 'Vicuña', 'Human (Tibetan)', 'Human (Sea Level)']
    altitudes = [0, 2500, 5000, 7500, 8500]
    
    fig = go.Figure()
    
    for species in species_list:
        sp = species_db[species]
        performances = []
        for alt in altitudes:
            max_alt = sp['max_altitude']
            if alt > max_alt * 1.5:
                perf = 0
            elif alt > max_alt:
                perf = 100 * np.exp(-((alt - max_alt) / 1000))
            else:
                perf = 100 - (alt / max_alt) * 25
            performances.append(max(0, perf))
        
        fig.add_trace(go.Scatter(
            x=altitudes, y=performances,
            mode='lines+markers',
            name=species,
            line=dict(width=3),
            marker=dict(size=10)
        ))
    
    # Reference lines
    fig.add_hline(y=50, line_dash="dash", line_color="orange")
    fig.add_vline(x=2500, line_dash="dot", line_color="gray",
                 annotation_text="Altitude Sickness")
    fig.add_vline(x=8848, line_dash="dot", line_color="gray",
                 annotation_text="Everest")
    
    fig.update_layout(
        title='<b>Altitude Performance Comparison</b>',
        xaxis_title='Altitude (m)',
        yaxis_title='Performance Capacity (%)',
        height=600,
        hovermode='x unified'
    )
    
    fig.show()
    
    print("\nKEY INSIGHTS:")
    print("• Bar-headed goose: >80% at 8,000m")
    print("• Vicuña: optimized for 3,000-5,500m")
    print("• Tibetan humans: genetic adaptations evident")
    print("• Sea-level humans: rapid decline >2,500m")

def compare_diving_species():
    """Compare all diving specialists"""
    species_list = ['Sperm Whale', 'Weddell Seal', 'Emperor Penguin', 'Human (Free Diver)']
    
    max_durs = []
    aero_lims = []
    o2_stores = []
    
    for species in species_list:
        sp = species_db[species]
        max_durs.append(sp['max_duration'])
        result = calculate_dive_oxygen(30, sp['mass_kg'], 
                                      sp['metabolic_rate'], sp['o2_stores'])
        aero_lims.append(result['aerobic_limit'])
        o2_stores.append(sp['o2_stores'])
    
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=('Max Duration', 'Aerobic Limit', 'O₂ Storage')
    )
    
    fig.add_trace(go.Bar(
        x=species_list, y=max_durs,
        marker_color='#0077B6',
        text=[f"{v:.0f}" for v in max_durs],
        textposition='auto'
    ), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=species_list, y=aero_lims,
        marker_color='#2A9D8F',
        text=[f"{v:.0f}" for v in aero_lims],
        textposition='auto'
    ), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=species_list, y=o2_stores,
        marker_color='#F4A261',
        text=[f"{v}%" for v in o2_stores],
        textposition='auto'
    ), row=1, col=3)
    
    fig.update_xaxes(tickangle=45)
    fig.update_yaxes(title_text="Minutes", row=1, col=1)
    fig.update_yaxes(title_text="Minutes", row=1, col=2)
    fig.update_yaxes(title_text="Percent", row=1, col=3)
    
    fig.update_layout(
        title_text='<b>Diving Performance Comparison</b>',
        height=500,
        showlegend=False
    )
    
    fig.show()
    
    print("\nKEY INSIGHTS:")
    print("• Sperm whale: 90min max (deepest, longest)")
    print("• Weddell seal: 82min (extreme bradycardia)")
    print("• Emperor penguin: 27min (bird + diving)")
    print("• Human: 11.5min (severely limited)")

# Create buttons
btn_alt = Button(
    description='🏔️ Compare Altitude',
    button_style='info',
    layout={'width': '200px'}
)
btn_alt.on_click(lambda b: compare_altitude_species())

btn_dive = Button(
    description='🐋 Compare Diving',
    button_style='info',
    layout={'width': '200px'}
)
btn_dive.on_click(lambda b: compare_diving_species())

display(HTML("<h3>📊 Multi-Species Comparisons</h3>"))
display(HBox([btn_alt, btn_dive]))

## Part 6: Challenge Problems

### Challenge 1: Everest Without Oxygen 🏔️

**Question**: Why can bar-headed geese fly at 8,848m while humans barely survive?

Use the altitude explorer to compare:
1. Bar-Headed Goose at 8,848m
2. Human (Sea Level) at 8,848m

Calculate:
- O₂ partial pressure at summit
- Performance capacity for each
- Difference in effective O₂ (with hemoglobin affinity)

<details>
<summary>Hint</summary>
Consider: respiratory efficiency (90% vs 25%), hemoglobin affinity (1.25× vs 1.0×), and metabolic demands of flight vs walking.
</details>

In [ ]:
# Challenge 1 workspace
print("CHALLENGE 1: EVEREST WITHOUT OXYGEN")
print("=" * 60)

everest = 8848
o2_everest = calculate_altitude_oxygen(everest)

print(f"\nMt. Everest Summit ({everest}m):")
print(f"O₂ partial pressure: {o2_everest:.1f} mmHg")
print(f"That's {o2_everest/160*100:.0f}% of sea level!\n")

# Compare goose vs human
goose = species_db['Bar-Headed Goose']
human = species_db['Human (Sea Level)']

goose_effective = o2_everest * goose['hb_affinity']
human_effective = o2_everest * human['hb_affinity']

print("BAR-HEADED GOOSE:")
print(f"  Effective O₂: {goose_effective:.1f} mmHg (with {goose['hb_affinity']:.2f}× Hb)")
print(f"  Respiratory efficiency: {goose['efficiency']}%")
print(f"  Status: FLYING actively!\n")

print("HUMAN (SEA LEVEL):")
print(f"  Effective O₂: {human_effective:.1f} mmHg (baseline Hb)")
print(f"  Respiratory efficiency: {human['efficiency']}%")
print(f"  Status: Barely conscious\n")

print("WHY THE DIFFERENCE?")
print(f"1. Hemoglobin: Goose {goose['hb_affinity']:.2f}× better O₂ binding")
print(f"2. Efficiency: {goose['efficiency']}% vs {human['efficiency']}% (3.6× more efficient!)")
print(f"3. Respiratory system: Unidirectional airflow vs bidirectional")
print(f"4. Metabolism: Goose maintains HIGH rate, human severely suppressed")

print("\nYOUR ANALYSIS:")
print("-" * 60)
# Add your thoughts here

### Challenge 2: Mars Habitat Design 🚀

**Question**: Design a breathable atmosphere for Mars colonists.

Mars facts:
- Surface pressure: 0.6% of Earth (4.5 mmHg)
- Humans need minimum 50 mmHg O₂ for survival
- Comfortable: 100+ mmHg O₂

Design options:
1. Earth-like (21% O₂, 760 mmHg total)
2. Reduced pressure (50% O₂, 300 mmHg total)
3. Pure O₂ at low pressure

Calculate trade-offs and recommend optimal design.

<details>
<summary>Hint</summary>
Consider: structural mass (pressure differential), fire risk (O₂ percentage), gas import costs, human comfort.
</details>

In [ ]:
# Challenge 2 workspace
print("CHALLENGE 2: MARS HABITAT ATMOSPHERE DESIGN")
print("=" * 60)

print("\nCONSTRAINTS:")
print("• Mars surface: 4.5 mmHg (unbreathable)")
print("• Minimum O₂: 50 mmHg (survival)")
print("• Comfortable O₂: 100+ mmHg\n")

options = {
    'Earth-like': {'total': 760, 'o2_percent': 21, 'o2_mmHg': 160},
    'Reduced': {'total': 300, 'o2_percent': 50, 'o2_mmHg': 150},
    'Pure O₂': {'total': 160, 'o2_percent': 100, 'o2_mmHg': 160}
}

print("DESIGN OPTIONS:\n")
for name, opt in options.items():
    print(f"{name}:")
    print(f"  Total pressure: {opt['total']} mmHg")
    print(f"  O₂ percentage: {opt['o2_percent']}%")
    print(f"  O₂ partial pressure: {opt['o2_mmHg']} mmHg")
    
    # Calculate equivalent altitude
    for alt in range(0, 10000, 100):
        if abs(calculate_altitude_oxygen(alt) - opt['o2_mmHg']) < 5:
            print(f"  Equivalent to Earth altitude: {alt}m")
            break
    print()

print("TRADE-OFF ANALYSIS:")
print("-" * 60)
print("Earth-like:")
print("  ✓ Most comfortable")
print("  ⚠ Heavy structure (760 mmHg pressure differential)")
print("  ⚠ More gas to import\n")

print("Reduced Pressure:")
print("  ✓ Lighter structure")
print("  ✓ Adequate O₂ (150 mmHg)")
print("  ⚠ Increased fire risk (50% O₂)")
print("  ⚠ Decompression issues\n")

print("Pure O₂:")
print("  ✓ Lightest structure")
print("  ⚠⚠ EXTREME fire hazard (Apollo 1!)")
print("  ⚠ Oxygen toxicity")
print("  ⚠ Material degradation\n")

print("YOUR RECOMMENDATION:")
print("-" * 60)
# What would you choose and why?

### Challenge 3: Whale vs Human Dive Competition 🐋

**Question**: Compare sperm whale and human free diver capabilities.

Use diving explorer to analyze:
1. Sperm whale at 60 minutes
2. Human free diver at 11.5 minutes (world record)

Calculate:
- O₂ storage per kg body mass
- Aerobic dive limits
- Key adaptations enabling whale performance

<details>
<summary>Hint</summary>
Compare: O₂ stores (90% vs 50%), myoglobin concentration (8× vs 1×), metabolic suppression (30% vs 70%).
</details>

In [ ]:
# Challenge 3 workspace
print("CHALLENGE 3: WHALE VS HUMAN DIVING")
print("=" * 60)

# Sperm whale analysis
whale = species_db['Sperm Whale']
whale_dive = calculate_dive_oxygen(60, whale['mass_kg'], 
                                   whale['metabolic_rate'], whale['o2_stores'])

# Human analysis
human = species_db['Human (Free Diver)']
human_dive = calculate_dive_oxygen(11.5, human['mass_kg'],
                                   human['metabolic_rate'], human['o2_stores'])

print("\nSPERM WHALE (60-minute dive):")
print(f"  Body mass: {whale['mass_kg']:,} kg")
print(f"  Total O₂: {whale_dive['total_o2']:,.0f} ml")
print(f"  O₂ per kg: {whale_dive['total_o2']/whale['mass_kg']:.1f} ml/kg")
print(f"  Aerobic limit: {whale_dive['aerobic_limit']:.1f} min")
print(f"  O₂ remaining: {whale_dive['o2_percent'][-1]:.0f}%")
print(f"  O₂ stores: {whale['o2_stores']}% (blood/muscle)")
print(f"  Myoglobin: {whale['myoglobin']:.0f}× terrestrial")
print(f"  Metabolism: {whale['metabolic_rate']*100:.0f}% (70% suppression!)\n")

print("HUMAN FREE DIVER (11.5-minute record):")
print(f"  Body mass: {human['mass_kg']} kg")
print(f"  Total O₂: {human_dive['total_o2']:.0f} ml")
print(f"  O₂ per kg: {human_dive['total_o2']/human['mass_kg']:.1f} ml/kg")
print(f"  Aerobic limit: {human_dive['aerobic_limit']:.1f} min")
print(f"  O₂ remaining: {human_dive['o2_percent'][-1]:.0f}%")
print(f"  O₂ stores: {human['o2_stores']}% (typical mammal)")
print(f"  Myoglobin: {human['myoglobin']:.0f}× (baseline)")
print(f"  Metabolism: {human['metabolic_rate']*100:.0f}% (limited suppression)\n")

print("COMPARISON:")
print("-" * 60)
duration_ratio = 60 / 11.5
storage_ratio = (whale_dive['total_o2']/whale['mass_kg']) / (human_dive['total_o2']/human['mass_kg'])

print(f"Duration: Whale dives {duration_ratio:.1f}× longer")
print(f"O₂/kg: Whale stores {storage_ratio:.1f}× more per kilogram")
print(f"Myoglobin: Whale has {whale['myoglobin']:.0f}× more muscle oxygen")
print(f"Metabolic suppression: Whale {100-whale['metabolic_rate']*100:.0f}% vs Human {100-human['metabolic_rate']*100:.0f}%")

print("\nKEY WHALE ADVANTAGES:")
print("1. O₂ storage: 90% in blood/muscle vs 50%")
print("2. Myoglobin: 8× concentration")
print("3. Bradycardia: 50%+ heart rate reduction")
print("4. Metabolism: 70% suppression")
print("5. Vasoconstriction: Blood to vital organs only")

print("\nYOUR ANALYSIS:")
print("-" * 60)
print("Which adaptation is MOST important? Why?")
# Add your reasoning

## Part 7: Export Your Work

In [ ]:
def export_results():
    """Export lab results"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = "/content"
    os.makedirs(output_dir, exist_ok=True)
    
    # Create summary
    data = []
    
    # Altitude species at various altitudes
    for species in ['Bar-Headed Goose', 'Human (Sea Level)']:
        for alt in [0, 2500, 5000, 8000]:
            sp = species_db[species]
            o2 = calculate_altitude_oxygen(alt)
            data.append({
                'Species': species,
                'Type': 'Altitude',
                'Altitude_m': alt,
                'O2_mmHg': f"{o2:.1f}",
                'Max_Altitude': sp['max_altitude']
            })
    
    # Diving species
    for species in ['Sperm Whale', 'Human (Free Diver)']:
        sp = species_db[species]
        result = calculate_dive_oxygen(30, sp['mass_kg'], 
                                      sp['metabolic_rate'], sp['o2_stores'])
        data.append({
            'Species': species,
            'Type': 'Diving',
            'Max_Duration': sp['max_duration'],
            'Aerobic_Limit': f"{result['aerobic_limit']:.1f}",
            'O2_Stores': f"{sp['o2_stores']}%"
        })
    
    df = pd.DataFrame(data)
    csv_file = f"{output_dir}/lab_6_3_results_{timestamp}.csv"
    df.to_csv(csv_file, index=False)
    print(f"✓ Results saved: {csv_file}")
    
    # Create summary figure
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Altitude Performance', 'Diving Capacity')
    )
    
    # Altitude plot
    alts = np.linspace(0, 10000, 50)
    for species in ['Bar-Headed Goose', 'Human (Sea Level)']:
        sp = species_db[species]
        perfs = []
        for a in alts:
            if a > sp['max_altitude'] * 1.5:
                perfs.append(0)
            elif a > sp['max_altitude']:
                perfs.append(100 * np.exp(-((a - sp['max_altitude']) / 1000)))
            else:
                perfs.append(100 - (a / sp['max_altitude']) * 25)
        
        fig.add_trace(go.Scatter(
            x=alts, y=perfs, name=species, line=dict(width=3)
        ), row=1, col=1)
    
    # Diving plot
    diving_sp = ['Sperm Whale', 'Weddell Seal', 'Emperor Penguin', 'Human (Free Diver)']
    durations = [species_db[s]['max_duration'] for s in diving_sp]
    
    fig.add_trace(go.Bar(
        x=diving_sp, y=durations, marker_color='#0077B6',
        text=[f"{d:.0f}" for d in durations], textposition='auto'
    ), row=1, col=2)
    
    fig.update_xaxes(title_text="Altitude (m)", row=1, col=1)
    fig.update_yaxes(title_text="Performance (%)", row=1, col=1)
    fig.update_yaxes(title_text="Max Duration (min)", row=1, col=2)
    fig.update_xaxes(tickangle=45, row=1, col=2)
    
    fig.update_layout(
        height=500,
        title_text='<b>Lab 6.3 Summary: Extreme Respiratory Performance</b>',
        showlegend=True
    )
    
    html_file = f"{output_dir}/lab_6_3_summary_{timestamp}.html"
    fig.write_html(html_file)
    print(f"✓ Summary saved: {html_file}")
    
    try:
        png_file = f"{output_dir}/lab_6_3_summary_{timestamp}.png"
        fig.write_image(png_file, width=1200, height=600)
        print(f"✓ PNG saved: {png_file}")
    except:
        print("ℹ PNG export requires kaleido (optional)")
    
    print("\n✓ Export complete!")

# Export button
export_btn = Button(
    description='📥 Export Results',
    button_style='success',
    icon='download',
    layout={'width': '200px'}
)
export_btn.on_click(lambda b: export_results())

display(HTML("<h3>📤 Export Your Work</h3>"))
display(export_btn)

## Summary & Reflection

### What You've Learned

✅ Environmental oxygen physics (altitude and diving)
✅ Performance limits across extreme conditions  
✅ Altitude specialists (bar-headed goose, vicuña, Tibetans)
✅ Diving champions (sperm whale, seals, penguins)
✅ Real-world applications (Everest, Mars, diving records)

### Key Insights

**Altitude:**
- O₂ drops exponentially with altitude (e^-altitude/7400)
- Bar-headed geese maintain 80%+ capacity at 8,000m
- Multiple adaptations work synergistically
- Humans limited without genetic adaptations

**Diving:**
- Sperm whales dive 90+ minutes to 2,000m
- 90% O₂ storage in blood/muscle (vs 50% terrestrial)
- Metabolic suppression critical (70% reduction)
- Humans severely limited vs marine mammals

**Universal Principles:**
- Oxygen availability determines limits
- Evolution finds diverse solutions
- Trade-offs between capabilities
- Quantitative analysis predicts performance

### Connection to Chapter 6

This lab integrated:
- Section 6.3: Lung evolution
- Section 6.4: Bird supremacy (bar-headed goose)
- Section 6.5: Diving adaptations (whale, seals)
- Section 6.6: High-altitude breathing

### Real-World Applications

- **Aviation**: Cabin pressure design
- **Space**: Life support systems
- **Medicine**: Hypoxia treatment
- **Sports**: High-altitude training
- **Conservation**: Climate vulnerability

### The Big Picture

From ocean depths to mountain peaks, vertebrates demonstrate remarkable respiratory adaptations. This lab shows:

- Pattern recognition: Similar challenges, diverse solutions
- Quantitative analysis: Performance can be predicted
- Evolutionary insight: Natural selection optimizes trade-offs
- Biomimetic potential: Nature inspires technology

**Congratulations on completing Lab 6.3!** 🎉

You now understand how vertebrates conquer Earth's most extreme respiratory challenges!